In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mglearn
%matplotlib inline
import seaborn as sns
import platform
from matplotlib import font_manager , rc

if platform.system() == 'Darwin':
  rc('font' , family = 'AppleGothic')
elif platform.system() == 'Windows':
  path = 'C:/Windows/Fonts/malgun.ttf'
  font_name = font_manager.FontProperties(fname = path).get_name()
  rc('font' , family = font_name)
else:
  print('모름')
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score , precision_score , recall_score , roc_auc_score , f1_score , confusion_matrix , roc_curve , precision_recall_curve

# 앙상블 학습(Ensemble learning
- 여러 개의 의사결정트리를 결합하여 하나의 결정트리보다 더 좋은 성능을 내는 머신러닝 학습기법
- 앙상블 학습의 핵심은 약한 분류기를 병렬(배깅) 또는 직렬(부스팅)로 결합하여 강렬한 분류기로 만드는 것이다
- 분류
 - 1. 배깅 : 동일 알고리즘을 병렬로 사용
 - 2. 부스팅 : 동일 알고리즘을 직렬로 사용
 - 3. 보팅 : 다른 알고리즘을 병렬로 사용

## 랜덤 포레스트(Random Forest)
- 앙상블 학습의 대표
- 결정트리를 랜덤하게 만들어 결정트리 숲을 만든다
- 랜덤 포레스트는 각각의 트리를 훈련하기 위한 데이터를 랜덤하게 만든다.
- 부트스트랩 샘플 : 부트스트랩 방식으로 샘플링하여 분류한 데이터
- 부트스트랩 샘플은 훈련세트의 크기와 같게 만든다.
- 랜덤 포레스트는 랜덤하게 선택된 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻는다.

In [3]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


- RandomForestClassifier : 분류
- RandomForestResressor : 회귀
- sklearn의 랜덤 포레스트는 100개의 결정트리를 훈련하는 방식
- 분류일 때 , 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스로 결과 예측
- 회귀일 때 , 각 트리의 예측값을 평균하여 결과 예측

In [4]:
data = wine.iloc[:,:-1].to_numpy()
target = wine.iloc[:,-1].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
train_input , test_input , train_target , test_target = train_test_split(data,target , test_size = 0.2 , random_state = 42)

In [9]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 42 , n_jobs = -1)
scores = cross_validate(rf , train_input , train_target , n_jobs = -1 , return_train_score = True)


In [15]:
np.mean(scores['train_score']) , np.mean(scores['test_score'])

(0.9973541965122431, 0.8905151032797809)

In [18]:
rf.fit(train_input , train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- 부트스트랩 샘플을 이용하여 학습하는 랜덤 포레스트는 데이터의 중복을 가져올 수 있고 , 어떤 데이터들은 학습에 사용되지 않을 수도 있다.
- 사용되지 않은 데이터들을 OOB라고 한다.
- OOB를 검증데이터로 활용하여 자체평가를 하는 기능을 제공

In [20]:
# oob_score = True
rf = RandomForestClassifier(oob_score = True , random_state = 42 , n_jobs = -1)
rf.fit(train_input , train_target)
print(rf.oob_score_)

0.8934000384837406


In [21]:
params = {
    'max_depth' : [8,16,24],
    'min_samples_leaf' : [1,6,12],
    'min_samples_split' : [2,8,16]
}

In [22]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(RandomForestClassifier(random_state = 42 , n_jobs = -1) , params , n_jobs = -1)
gs.fit(train_input , train_target)

GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [8, 16, 24],
                         'min_samples_leaf': [1, 6, 12],
                         'min_samples_split': [2, 8, 16]})

In [23]:
print(gs.best_params_)
print(gs.best_score_)

{'max_depth': 24, 'min_samples_leaf': 1, 'min_samples_split': 2}
0.8909000888428222


In [24]:
gs.cv_results_

{'mean_fit_time': array([0.31637807, 0.49762564, 0.48652892, 0.42002258, 0.40280976,
        0.44655704, 0.44958658, 0.47371135, 0.44021354, 0.56850357,
        0.57852712, 0.41934438, 0.48796887, 0.42712636, 0.50755711,
        0.46538119, 0.48225665, 0.48020735, 0.53366027, 0.60605879,
        0.54689341, 0.53192525, 0.51488032, 0.45544939, 0.4574059 ,
        0.42362938, 0.42697515]),
 'std_fit_time': array([0.00904787, 0.05981696, 0.06564554, 0.07992005, 0.12008337,
        0.05954807, 0.05583306, 0.09137177, 0.08169579, 0.12009062,
        0.11198667, 0.05341432, 0.10829817, 0.09681555, 0.09821002,
        0.13054783, 0.13023188, 0.11251728, 0.12121307, 0.04749016,
        0.19318612, 0.10566838, 0.16769812, 0.1592032 , 0.10009625,
        0.11493718, 0.10183488]),
 'mean_score_time': array([0.03790879, 0.09263921, 0.06224122, 0.08672476, 0.05543532,
        0.05740957, 0.07437944, 0.06155105, 0.05842071, 0.15021415,
        0.09550033, 0.11855006, 0.11089625, 0.059163  , 0.094779

## 보팅(Voting Classifier)
- 서로 다른 알고리즘을 병렬로 학습시켜 최적의 값을 도출

In [26]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression , LinearRegression , Ridge , Lasso

In [36]:
lr = LogisticRegression()
knn = KNeighborsClassifier()

#소프트 보팅

vo = VotingClassifier(estimators = [('LR' , lr),('Knn' , knn)] , voting = 'soft')

#학습

vo.fit(train_input , train_target)

#예측

pred = vo.predict(test_input)

#정확도

accuracy = accuracy_score(test_target , pred)

print('voting 분류기 정확도 : ' , accuracy)

models = [lr,knn]
for i in models:
    i.fit(train_input , train_target)
    pred = i.predict(test_input)
    model_name = i.__class__.__name__
    score = accuracy_score(test_target , pred)
    print(f'{model_name}의 정확도 : {score}')

voting 분류기 정확도 :  0.8630769230769231
LogisticRegression의 정확도 : 0.7753846153846153
KNeighborsClassifier의 정확도 : 0.8553846153846154
